In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import sqlalchemy
import os
from sqlalchemy import create_engine
from sqlalchemy import text
from datetime import datetime, timedelta
from math import floor

import sys
sys.path.insert(0, r'C:\Scripts\modules\database')
sys.path.insert(0, r'C:\Scripts\modules\parameters')

from connection import Connect
from bases import Bases


diretorio_base = "C:\\Scripts\\relatórios\\"

In [ ]:
conexao = Connect.connect_techdb()

# Execute and get the single value
result = conexao.execute(
    text('SELECT MAX("Data") FROM principal."pl_historico_diario";')
).scalar()

conexao.close()

# Convert to pandas Timestamp only if result is not None
if result:
    dt_max = pd.to_datetime(result)
else:
    dt_max = None 
    print("No data found in table.")

In [ ]:
dt_max

In [ ]:
pastas_meses = []
pastas_dias = []
for pasta_mes in next(os.walk(diretorio_base))[1]:
    if pasta_mes[0] == "m":
        pastas_meses.append(pasta_mes)
for mes in pastas_meses:
    for dia in next(os.walk("{}{}".format(diretorio_base, mes)))[1]:
        pastas_dias.append({"mes":mes, "dia":dia})
pastas_dias.sort(key = lambda date: datetime.strptime(date["dia"], "%d-%m-%Y"))
pastas_dias = [pasta for pasta in pastas_dias if datetime.strptime(pasta["dia"], "%d-%m-%Y") > dt_max]

In [ ]:
import os
import pandas as pd
from datetime import datetime

# 1. Define a data de hoje para buscar a pasta
data_hoje_str = datetime.now().strftime("%d-%m-%Y")

# Caminho: C:\Scripts\relatórios\07-01-2026\
diretorio_dia = os.path.join(diretorio_base, data_hoje_str)

try:
    # 2. Busca a pasta de "horário" (ex: 103005 ou qualquer subpasta que estiver lá)
    subpastas = [f for f in os.listdir(diretorio_dia) if os.path.isdir(os.path.join(diretorio_dia, f))]
    
    if subpastas:
        # Pega a primeira pasta de horário encontrada
        pasta_horario = subpastas[0]
        caminho_arquivo = os.path.join(diretorio_dia, pasta_horario, "base_btg.xlsx")
        
        print(f"Lendo: {caminho_arquivo}")

        # 3. Leitura do Excel
        try:
            temp_df = pd.read_excel(caminho_arquivo, header=2)
            if "Conta" not in temp_df.columns:
                raise ValueError
        except:
            temp_df = pd.read_excel(caminho_arquivo)

        # 4. Tratamento dos dados
        temp_df.columns = [str(c).replace(" (R$)", "") for c in temp_df.columns]
        
        colunas_finais = ["Conta", "Assessor", "PL Total", "Tipo", "Conta Corrente", 
                          "Fundos", "Renda Fixa", "Renda Variável", "Previdência", "Derivativos"]
        
        pl_historico = temp_df[[c for c in colunas_finais if c in temp_df.columns]].copy()
        
        # Força a data da coluna para ser a data da pasta (hoje)
        pl_historico['Data'] = pd.to_datetime(data_hoje_str, format="%d-%m-%Y")
        pl_historico['Mês'] = pl_historico['Data'].dt.strftime("%Y-%m")
        pl_historico['Conta'] = pl_historico['Conta'].astype(str)

        print(f"Sucesso! {len(pl_historico)} linhas carregadas.")
        
    else:
        print(f"Nenhuma subpasta de horário encontrada em: {diretorio_dia}")
        pl_historico = pd.DataFrame()

except FileNotFoundError:
    print(f"Pasta do dia {data_hoje_str} não encontrada.")
    pl_historico = pd.DataFrame()

In [ ]:

conexao = Connect.connect_techdb()
pl_historico.to_sql('pl_historico_diario', 
                    con=conexao, 
                    index=False, 
                    if_exists='append',
                    schema="principal")
conexao.close()